In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q librosa
!pip install -q lightgbm
import pandas as pd
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set Paths
import os
BASE_PATH = "/content/drive/MyDrive/SHL_Grammar_Scoring"
AUDIO_DIR_TRAIN = os.path.join(BASE_PATH, "audios_train")
TRANSCRIPT_PATH = os.path.join(BASE_PATH, "transcripts.csv")


# Load Whisper Model
from transformers import pipeline
import torch

asr = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-large-v3",
    device=0 if torch.cuda.is_available() else -1,
    return_timestamps=True
)



  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 32.0 MB

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:

#Transcribe All Audio Files

from tqdm import tqdm
import pandas as pd

transcripts = {}
for idx, filename in enumerate(tqdm(os.listdir(AUDIO_DIR_TRAIN))):
    if filename.endswith(".wav"):
        file_path = os.path.join(AUDIO_DIR_TRAIN, filename)
        try:
            result = asr(file_path)
            text = result["text"]
            transcripts[filename] = text

            #Print immediately
            print(f"\n {idx+1}. File: {filename}")
            print(f" Transcript: {text}")
            print("-" * 60)

        except Exception as e:
            print(f"Error transcribing {filename}: {e}")
            transcripts[filename] = ""

# Convert to DataFrame
df_transcripts = pd.DataFrame(list(transcripts.items()), columns=["file", "transcript"])


# Print Every 10th Transcript
for idx in range(0, len(df_transcripts), 10):
    file = df_transcripts.iloc[idx]["file"]
    text = df_transcripts.iloc[idx]["transcript"]
    print(f"🎧 File: {file}\n📝 Transcript: {text}\n{'-'*50}")

# Save Transcripts to Google Drive
df_transcripts.to_csv(TRANSCRIPT_PATH, index=False)
print(f"✅ Transcripts saved to: {TRANSCRIPT_PATH}")


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Device set to use cuda:0
  0%|          | 0/444 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
  0%|          | 1/444 [00:05<37:44,  5.11s/it]/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(



📁 1. File: audio_1024.wav
📝 Transcript:  My favorite place is in Andhra Pradesh. It is in Chittoor district. It is the temple of Lord Venkateshwara Swami. It has seven hills to visit the temple. It is a good place to receive the Tirumala for India year. Crores of people to visit the Lord Venkateshwara Swami. In Tirumala have a lot of places to visit the tourists. In Tirumala, Laddu is very famous.
------------------------------------------------------------


Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
  0%|          | 2/444 [00:10<37:05,  5.04s/it]/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(



📁 2. File: audio_1030.wav
📝 Transcript:  My favorite place is Tikal hostel. The journey or travel experience is very good. And that places you can never forgot that food is dosa. that it is very delicious and also that to my flower also can sit in side b me side b and also we can eat together or on particular time of day or season when this place is specially magical or beautiful but night time we will go that place so
------------------------------------------------------------


...


📁 442. File: audio_993.wav
📝 Transcript:  Kepala Kepala Kepala Kepala Kepala Kepala Kepala Kepala Kepala Kepala Kepala Kepala Kepala Kepala Hai, kawa. Tua mawawar yuk, atas tuwa kawang. Tawa ta, me, atas tuwa. Sampai jumpa. Sampai jumpa. Sampai jumpa. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Sipta. Terima kasih telah menonton!
------------------------------------------------------------


Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
100%|█████████▉| 443/444 [1:12:00<00:11, 11.93s/it]/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(



📁 443. File: audio_990.wav
📝 Transcript:  There is a playground the size of four football fields. It's two football fields long and two football fields wide. At this playground there are a basketball court, there is a tennis court, and there is also a playground for the kids that has a slide, a jungle gym, gym and different pathways for them to walk around and make them think like they're marching or camping or something. I'm not really sure. The things I hear around this playground are cars because the playground is on an island, even though, like an island where there are a highway, a road on one side, a road on the other. It's near a school, so there's always school buses going by and cars, and there's always kids at the playground that always laugh and it's a very clean playground.
------------------------------------------------------------


100%|██████████| 444/444 [1:12:07<00:00,  9.75s/it]


📁 444. File: audio_994.wav
📝 Transcript:  The playground that I'm imagining is quite big, full of slides and swings that are very colorful, very geared towards small or little children. There are plenty of things to do, you know, because they have lines, they have the swings. They also have areas where they can run, where they can talk. I can see a couple of sandboxes that they can use, and that's all you see, some plastic shovels, plastic buckets, maybe some toys that they can use to play around. I also see plenty of basketballs or soccer balls that they can use to play in certain areas. I can hear laughter, I can see trees, and I can hear the birds.
------------------------------------------------------------
🎧 File: audio_1024.wav
📝 Transcript:  My favorite place is in Andhra Pradesh. It is in Chittoor district. It is the temple of Lord Venkateshwara Swami. It has seven hills to visit the temple. It is a good place to receive the Tirumala for India year. Crores of people to visit t

In [ ]:
import re
import string

def clean_transcript(text):
    text = text.lower()
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)  # remove punctuation
    text = re.sub(r'\s+', ' ', text)  # normalize whitespace
    return text.strip()


In [ ]:
import pandas as pd
import os

# Path to your saved transcripts CSV in Drive
transcript_path = os.path.join(BASE_PATH, "transcripts.csv")  # or "test_transcripts.csv"

# Load saved transcripts
df_transcripts = pd.read_csv(transcript_path)

# Standardize filenames
df_transcripts["file"] = df_transcripts["file"].str.strip().str.lower()


# Apply updated cleaner
df_transcripts["cleaned"] = df_transcripts["transcript"].fillna("").apply(clean_transcript)

# Drop any empty results
df_transcripts = df_transcripts[df_transcripts["cleaned"].str.strip().astype(bool)]

# Preview
df_transcripts.head()


,file,transcript,cleaned
0,audio_1024.wav,My favorite place is in Andhra Pradesh. It is...,my favorite place is in andhra pradesh it is i...
1,audio_1030.wav,My favorite place is Tikal hostel. The journe...,my favorite place is tikal hostel the journey ...
2,audio_1025.wav,I would like to go the Saachi for the Saachi ...,i would like to go the saachi for the saachi s...
3,audio_1028.wav,My favorite place to visit Kerala is so beaut...,my favorite place to visit kerala is so beauti...
4,audio_1008.wav,The school playground is lively and filled wi...,the school playground is lively and filled wit...


In [ ]:
# Load train.csv
df_scores = pd.read_csv(os.path.join(BASE_PATH, "train.csv"))

# Rename 'filename' to 'file' in df_scores to match df_transcripts
df_scores.rename(columns={"filename": "file"}, inplace=True)

# Normalize both columns (optional but safer)
df_scores["file"] = df_scores["file"].str.lower().str.strip()
df_transcripts["file"] = df_transcripts["file"].str.lower().str.strip()

# Merge
df_merged = pd.merge(df_scores, df_transcripts, on="file", how="inner")

print(" Merged shape:", df_merged.shape)
df_merged.head()


 Merged shape: (444, 4)


,file,label,transcript,cleaned
0,audio_1261.wav,1.0,Takk for ating med. My favorite hobby is cult...,takk for ating med my favorite hobby is cultiv...
1,audio_942.wav,1.5,The playground looks like very clear and neat...,the playground looks like very clear and neat ...
2,audio_1110.wav,1.5,My goal is to become an electrical employee a...,my goal is to become an electrical employee an...
3,audio_1024.wav,1.5,My favorite place is in Andhra Pradesh. It is...,my favorite place is in andhra pradesh it is i...
4,audio_538.wav,2.0,My favorite place is Ooty and Kodaikanal. My ...,my favorite place is ooty and kodaikanal my ex...


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load Sentence-BERT model
embedder = SentenceTransformer("paraphrase-mpnet-base-v2")

# Generate embeddings from cleaned transcript text
X_embed = embedder.encode(
    df_merged["cleaned"].tolist(),
    convert_to_numpy=True,
    show_progress_bar=True
)

# Grammar scores as target
y = df_merged["label"].values

print("Embedding shape:", X_embed.shape)
print("Target shape:", y.shape)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/14 [00:00<?, ?it/s]

Embedding shape: (444, 768)
Target shape: (444,)


In [ ]:
!pip install -q language-tool-python textstat spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 115.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import textstat
import spacy
import language_tool_python

# Initialize tools
tool = language_tool_python.LanguageTool('en-US')
nlp = spacy.load("en_core_web_sm")

def extract_features(text):
    features = {}
    features["flesch_reading_ease"] = textstat.flesch_reading_ease(text)
    features["sentence_count"] = textstat.sentence_count(text)
    features["grammar_errors"] = len(tool.check(text))

    # spaCy-based features
    doc = nlp(text)
    features["token_count"] = len(doc)
    features["avg_token_length"] = sum(len(token.text) for token in doc) / len(doc) if len(doc) > 0 else 0
    features["noun_count"] = len([token for token in doc if token.pos_ == "NOUN"])
    features["verb_count"] = len([token for token in doc if token.pos_ == "VERB"])
    features["adjective_count"] = len([token for token in doc if token.pos_ == "ADJ"])
    return features


INFO:language_tool_python.download_lt:Unzipping /tmp/tmpqli_r0xf.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-6.5.zip to /root/.cache/language_tool_python.


In [ ]:
# Apply to cleaned transcripts
linguistic_features_df = df_merged["cleaned"].apply(extract_features).apply(pd.Series)

# Check a few
linguistic_features_df.head()


,flesch_reading_ease,sentence_count,grammar_errors,token_count,avg_token_length,noun_count,verb_count,adjective_count
0,21.40,1.0,6.0,41.0,4.878049,12.0,6.0,2.0
1,55.24,1.0,4.0,41.0,4.292683,10.0,6.0,4.0
2,9.22,1.0,5.0,53.0,4.698113,9.0,10.0,4.0
3,21.41,1.0,10.0,66.0,4.318182,14.0,6.0,3.0
4,-30.37,1.0,18.0,117.0,4.222222,23.0,7.0,17.0


In [ ]:
X_structured = linguistic_features_df.fillna(0).to_numpy()
X_combined = np.hstack([X_embed, X_structured])


In [ ]:
import numpy as np

# Convert structured features to numpy
X_structured = linguistic_features_df.fillna(0).to_numpy()

# Combine embeddings with handcrafted features
X_combined = np.hstack([X_embed, X_structured])

print(" Combined feature shape:", X_combined.shape)


 Combined feature shape: (444, 776)


In [ ]:
!pip install -q xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.3/201.3 MB 2.8 MB/s eta 0:00:00


In [ ]:
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBRegressor, callback
from lightgbm import LGBMRegressor, early_stopping, log_evaluation
from sklearn.model_selection import KFold
from scipy.stats import pearsonr
import numpy as np

kf = KFold(n_splits=10, shuffle=True, random_state=42)

xgb_models = []
lgbm_models = []
xgb_scores = []
lgbm_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_combined)):
    print(f"\nFold {fold + 1}")

    X_train, X_val = X_combined[train_idx], X_combined[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
#used optuna to find out the best parameters
    xgb_model= XGBRegressor(
        n_estimators= 366,
        max_depth= 3,
        learning_rate = 0.031027775103966572,
        min_child_weight= 7,
        gamma = 0.6098090233808116,
        subsample= 0.532343732922544,
        colsample_bytree= 0.7975102153963436,
        reg_alpha= 0.6395032212192129,
        reg_lambda= 0.23416323808855305,
        objective = "reg:squarederror",
        random_state = 42,
        early_stopping_rounds=50,
    )

    xgb_model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],

        verbose=False
    )

    xgb_preds = xgb_model.predict(X_val)
    xgb_pcc, _ = pearsonr(y_val, xgb_preds)
    xgb_models.append(xgb_model)
    xgb_scores.append(xgb_pcc)

    print(f" XGBoost Pearson: {xgb_pcc:.4f}")

    # LightGBM Model
    lgbm_model = LGBMRegressor(
        n_estimators= 1000,
        learning_rate= 0.03480084659658307,
        max_depth= 4,
        num_leaves= 52,
        min_child_samples= 9,
        subsample= 0.768831773944386,
        colsample_bytree= 0.742555540221322,
        reg_alpha= 0.06782282648271094,
        reg_lambda= 0.40322645239417965,
        objective= "regression",
        random_state= 42
      )

    lgbm_model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        callbacks=[
            lgb.early_stopping(stopping_rounds=50),  # Corrected LightGBM syntax
            lgb.log_evaluation(period=100)           # Corrected LightGBM syntax
        ]
    )
    lgbm_preds = lgbm_model.predict(X_val)
    lgbm_pcc, _ = pearsonr(y_val, lgbm_preds)
    lgbm_models.append(lgbm_model)
    lgbm_scores.append(lgbm_pcc)

    print(f" LightGBM Pearson: {lgbm_pcc:.4f}")


Fold 1
 XGBoost Pearson: 0.5727
Training until validation scores don't improve for 50 rounds
[100]	valid_0's l2: 0.821913
[200]	valid_0's l2: 0.784659
[300]	valid_0's l2: 0.774859
[400]	valid_0's l2: 0.766397
[500]	valid_0's l2: 0.762571
Early stopping, best iteration is:
[543]	valid_0's l2: 0.76148
 LightGBM Pearson: 0.6548

Fold 2


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


 XGBoost Pearson: 0.7069
Training until validation scores don't improve for 50 rounds
[100]	valid_0's l2: 0.83656
[200]	valid_0's l2: 0.75828
[300]	valid_0's l2: 0.740637
[400]	valid_0's l2: 0.734798
[500]	valid_0's l2: 0.733238
Early stopping, best iteration is:
[481]	valid_0's l2: 0.732947


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


 LightGBM Pearson: 0.7161

Fold 3
 XGBoost Pearson: 0.7067
Training until validation scores don't improve for 50 rounds
[100]	valid_0's l2: 0.725222
[200]	valid_0's l2: 0.687777
[300]	valid_0's l2: 0.673762
Early stopping, best iteration is:
[291]	valid_0's l2: 0.6725


...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


 XGBoost Pearson: 0.6611
Training until validation scores don't improve for 50 rounds
[100]	valid_0's l2: 0.699012
[200]	valid_0's l2: 0.67604
Early stopping, best iteration is:
[177]	valid_0's l2: 0.673433
 LightGBM Pearson: 0.7320

Fold 10


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


 XGBoost Pearson: 0.6036
Training until validation scores don't improve for 50 rounds
[100]	valid_0's l2: 0.761527
[200]	valid_0's l2: 0.740299
[300]	valid_0's l2: 0.727954
Early stopping, best iteration is:
[347]	valid_0's l2: 0.726272
 LightGBM Pearson: 0.6341


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [ ]:
import joblib
import os

# Create a directory to store models
model_dir = os.path.join(BASE_PATH, "models")
os.makedirs(model_dir, exist_ok=True)

# Save XGBoost models
for i, model in enumerate(xgb_models):
    path = os.path.join(model_dir, f"xgb_model_fold{i+1}.pkl")
    joblib.dump(model, path)

# Save LightGBM models
for i, model in enumerate(lgbm_models):
    path = os.path.join(model_dir, f"lgbm_model_fold{i+1}.pkl")
    joblib.dump(model, path)

print(f" Saved all models to: {model_dir}")


 Saved all models to: /content/drive/MyDrive/SHL_Grammar_Scoring/models


In [ ]:
AUDIO_DIR_TEST = os.path.join(BASE_PATH, "audios_test")


In [ ]:
from tqdm import tqdm
test_transcripts = {}

for idx, filename in enumerate(tqdm(os.listdir(AUDIO_DIR_TEST))):
    if filename.endswith(".wav"):
        file_path = os.path.join(AUDIO_DIR_TEST, filename)
        try:
            result = asr(file_path)
            text = result["text"]
            test_transcripts[filename] = text
            print(f"\n File: {filename}\n Transcript: {text}\n" + "-" * 60)
        except Exception as e:
            print(f" Error: {filename} - {e}")
            test_transcripts[filename] = ""


  0%|          | 0/186 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
  1%|          | 1/186 [00:05<15:54,  5.16s/it]


 File: audio_1061.wav
 Transcript:  My travel journey experience destination is in Tagaytay. Some of it is very cool and very special place to me. I like the food, local food, bulalo. And the dish is so very... Very good. The taste is good. The place is very good. The particular time and day I spend the sunny days. Because it's a sunny days to enjoy walking around and see the view. Thank you for watching!
------------------------------------------------------------


/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
  1%|          | 2/186 [00:09<13:47,  4.50s/it]/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(



 File: audio_10.wav
 Transcript:  The playground was a large area made with cement with a big pergola on top covering the sun making it have a lot of shade. I noticed on the floor there was lines made to play handball. There were also a basketball court and a netball court. were popular amongst the children. This sounds like he was a lawnmower going and children screaming and playing.
------------------------------------------------------------


Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
  2%|▏         | 3/186 [00:14<14:15,  4.68s/it]

...


 File: audio_932.wav
 Transcript:  Actually, in my case, the happiest day of life is different from everyone because it can be anything from small to big or big to small. In my case, it was the day when I scored the highest in my class. The happiest day of my life was full of joy and fun. Everyone was praising me. My friends and my family were proud of me for doing something that no one believed I could achieve. Even it was so unbelievable for myself too. The day my test results were out, my life filled with joy. I was doing poorly in my studies, scoring average marks. However, my father taught me to never give up. So I kept studying.
------------------------------------------------------------


/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
 99%|█████████▉| 185/186 [26:35<00:11, 11.20s/it]


 File: audio_908.wav
 Transcript:  میری زندگی کے بہترین دن ہے۔ ہم مانالی سے جاتے ہیں، دو دنوں کے ساتھ۔ میری زندگی کے بہترین دن ہے جب ہم روتاپاس پہنچے گئے۔ ہم نے بہت سے سب سے سوچا ہے۔ اور میں آپ کو جو بھی ہیں جو جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی ہیں جو بھی 

/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
100%|██████████| 186/186 [26:41<00:00,  8.61s/it]


 File: audio_922.wav
 Transcript:  Звук відео Найкращий день в моєму житті був день, коли я отримав адміністраційну адміністрацію. Я спробував багато інших спроб, але це не було виступним. Але втім, на цей день, я побачив на листі моїх адміністраційних студентів. І це був курс, який я хотів стати. І це було дуже-дуже екстатично, будь-яке я згадаю.
------------------------------------------------------------


In [ ]:
import pandas as pd
import re
import string

def clean_transcript(text):
    text = text.lower()
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)  # remove punctuation
    text = re.sub(r'\s+', ' ', text)  # normalize whitespace
    return text.strip()
# Convert transcript dict to DataFrame
df_test = pd.DataFrame(list(test_transcripts.items()), columns=["file", "transcript"])

# Clean transcripts
df_test["cleaned"] = df_test["transcript"].fillna("").apply(clean_transcript)

# Drop empty cleaned transcripts
df_test = df_test[df_test["cleaned"].str.strip().astype(bool)]

# Preview
df_test.head()


,file,transcript,cleaned
0,audio_1061.wav,My travel journey experience destination is i...,my travel journey experience destination is in...
1,audio_10.wav,The playground was a large area made with cem...,the playground was a large area made with ceme...
2,audio_1035.wav,I would like to visit South Korea because I r...,i would like to visit south korea because i re...
3,audio_1012.wav,The school playground is lively and filled wi...,the school playground is lively and filled wit...
4,audio_1054.wav,my favorite place is Joghpaz and Gokarna and ...,my favorite place is joghpaz and gokarna and m...


In [ ]:
# Generate sentence embeddings for test set
from sentence_transformers import SentenceTransformer
import numpy as np

# Load Sentence-BERT model
embedder = SentenceTransformer("paraphrase-mpnet-base-v2")

X_test_embed = embedder.encode(
    df_test["cleaned"].tolist(),
    convert_to_numpy=True,
    show_progress_bar=True
)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
# Apply feature extraction to test set
test_linguistic_features_df = df_test["cleaned"].apply(extract_features).apply(pd.Series)

# Fill any missing values
test_linguistic_features_df = test_linguistic_features_df.fillna(0)

# Preview
test_linguistic_features_df.head()


,flesch_reading_ease,sentence_count,grammar_errors,token_count,avg_token_length,noun_count,verb_count,adjective_count
0,16.33,1.0,9.0,71.0,4.042254,17.0,7.0,9.0
1,22.42,1.0,3.0,65.0,4.369231,18.0,12.0,4.0
2,5.17,1.0,10.0,83.0,4.168675,18.0,12.0,3.0
3,6.86,1.0,4.0,72.0,5.402778,24.0,10.0,4.0
4,-47.28,1.0,20.0,145.0,3.634483,18.0,14.0,13.0


In [ ]:
import numpy as np

# Convert handcrafted features to NumPy array
X_test_structured = test_linguistic_features_df.to_numpy()

# Combine with sentence embeddings
X_test_combined = np.hstack([X_test_embed, X_test_structured])

# Check shape
print("Final test feature shape:", X_test_combined.shape)


Final test feature shape: (186, 776)


In [ ]:
model_dir = os.path.join(BASE_PATH, "models")


In [ ]:
import joblib
import os

model_dir = os.path.join(BASE_PATH, "models")

# Load XGBoost models
xgb_models = []
for i in range(10):
    path = os.path.join(model_dir, f"xgb_model_fold{i+1}.pkl")
    xgb_models.append(joblib.load(path))

# Load LightGBM models
lgbm_models = []
for i in range(10):
    path = os.path.join(model_dir, f"lgbm_model_fold{i+1}.pkl")
    lgbm_models.append(joblib.load(path))

print(" All models successfully loaded from Drive.")


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [04:12:52] WARNING: /workspace/src/collective/../data/../common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)


 All models successfully loaded from Drive.


In [ ]:
# Predict using all XGBoost folds
xgb_test_preds = np.mean([model.predict(X_test_combined) for model in xgb_models], axis=0)

# Predict using all LightGBM folds
lgbm_test_preds = np.mean([model.predict(X_test_combined) for model in lgbm_models], axis=0)

# Final ensemble (average) # didnot use since lightgbm alone was performing better than the ensemble
ensemble_preds = (xgb_test_preds + lgbm_test_preds) / 2


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

In [ ]:
import pandas as pd
import os

# Create DataFrame with predictions
df_submission = pd.DataFrame({
    "file": df_test["file"].str.strip().str.lower(),
    "label": lgbm_test_preds
})

# Load metadata
df_text = pd.read_csv(os.path.join(BASE_PATH, "test.csv"))
df_text["filename"] = df_text["filename"].str.strip().str.lower()

# Merge and fill missing values with 2.5
df_final = df_text.merge(df_submission, left_on="filename", right_on="file", how="left")
df_final.drop(columns=["file"], inplace=True)
df_final["label"] = df_final["label"].fillna(2.5)
df_final.rename(columns={"filename": "filename", "label": "label"}, inplace=True)


Submitted By Vishesh Gupta
(visheshgupta14x@gmail.com)